In [1]:
import numpy as np
import math

In [2]:
from numpy import matmul as mm
from numpy.linalg import inv as inv

\begin{align*}
\text{roll: }R_x(\phi) &= \begin{pmatrix}1&0&0\\0& cos\phi & -sin\phi \\0&sin\phi&cos\phi\end{pmatrix} \quad \text{pitch: }R_y(\theta) = \begin{pmatrix}cos\theta &0 & sin\theta\\0&1&0 \\-sin\theta&0&cos\theta\end{pmatrix} \\
&\text{yaw: }R_z(\psi) = \begin{pmatrix}cos\psi & -sin\psi &0 \\sin\psi&cos\psi&0\\0&0&1\end{pmatrix}
\end{align*}

In [3]:
def Rx(angle): # degree
    angle = angle/180*np.pi
    return np.array([[1,0,0],
                     [0,np.cos(angle),-np.sin(angle)],
                     [0,np.sin(angle),np.cos(angle)]])

In [4]:
Rx(180)

array([[ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00, -1.0000000e+00, -1.2246468e-16],
       [ 0.0000000e+00,  1.2246468e-16, -1.0000000e+00]])

In [5]:
def Ry(angle):
    angle = angle/180*np.pi
    return np.array([[np.cos(angle),0,np.sin(angle)],
                     [0,1,0],
                     [-np.sin(angle),0,np.cos(angle)]])
def Rz(angle):
    angle = angle/180*np.pi
    return np.array([[np.cos(angle),-np.sin(angle),0],
                     [np.sin(angle),np.cos(angle),0],
                     [0,0,1]])
print(Ry(180))
print(Rz(180))

[[-1.0000000e+00  0.0000000e+00  1.2246468e-16]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00]
 [-1.2246468e-16  0.0000000e+00 -1.0000000e+00]]
[[-1.0000000e+00 -1.2246468e-16  0.0000000e+00]
 [ 1.2246468e-16 -1.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  1.0000000e+00]]


In [6]:
def R(roll,pitch,yaw):
    return mm(mm(Rz(yaw),Ry(pitch)),Rx(roll))

print(R(0,0,0))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [7]:
def T(tx,ty,tz):
    return np.array([[tx],[ty],[tz]])

In [8]:
print(T(2,0,1))

[[2]
 [0]
 [1]]


### Rigid body transformation

In [9]:
def Pr(roll,pitch,yaw,tx,ty,tz):
    Pr_ = np.zeros((3,4))
    Pr_[:3,:3] = R(roll,pitch,yaw)
    Pr_[:,3:] = T(tx,ty,tz)
    print(Pr_)
    return Pr_

In [10]:
P = Pr(0,0,0,0,0,0)
print(P)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


### Perspective projection

In [11]:
def Pp(f):
    return np.array([[f,0,0],[0,f,0],[0,0,1]])
print(Pp(.1))

[[0.1 0.  0. ]
 [0.  0.1 0. ]
 [0.  0.  1. ]]


### CCD Imaging

In [12]:
def Pc(ku,kv,u0,v0):
    return np.array([[ku,0,u0],[0,kv,v0],[0,0,1]])

In [13]:
print(Pc(5,2,0.1,0.2))

[[5.  0.  0.1]
 [0.  2.  0.2]
 [0.  0.  1. ]]


In [14]:
dct = {'roll':0,'pitch':0,'yaw':0,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
print(dct['roll'])

0


In [15]:
def P(dct):
    Pr_ = Pr(dct['roll'],dct['pitch'],
            dct['yaw'],dct['tx'],dct['ty'],dct['tz'])
    Pp_ = Pp(dct['f'])
    Pc_ = Pc(dct['ku'],dct['kv'],dct['u0'],dct['v0'])
    return mm(Pc_,Pp_),mm(mm(Pc_,Pp_),Pr_)

In [16]:
M,P_ = P(dct)
print(P_)
print(M)

[[1. 0. 0. 2.]
 [0. 1. 0. 1.]
 [0. 0. 1. 0.]]
[[0.5 0.  0.1 1. ]
 [0.  0.2 0.2 0.2]
 [0.  0.  1.  0. ]]
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]


### Decomposition

In [17]:
def correctK(matrix):
    Knew = matrix.copy()
    for i in range(Knew.shape[0]):
        for j in range(Knew.shape[1]):
            if str(Knew[i,j]).find('-') != -1:
                Knew[i,j] = abs(Knew[i,j])
            if Knew[i,j] < 1e-10 and Knew[i,j] > -1e-10:
                Knew[i,j] = 0
    return Knew

In [18]:
def correctR(matrix):
    Rnew = matrix.copy()
    for i in range(Rnew.shape[0]):
        for j in range(Rnew.shape[1]):
            if Rnew[i,j]<1e-10 and Rnew[i,j]>-1e-10:
                Rnew[i,j]=0
    return Rnew

In [19]:
def P2KRT(matrix):
    left = matrix[:,:3]
    right = matrix[:,3:]
    q,r = np.linalg.qr(inv(left))
    k = correctK(inv(r))
    rot = np.linalg.solve(k,left)
    rot = correctR(rot)
    trl = np.linalg.solve(k,right)
    return rot,k,trl

In [20]:
def checkProcessP2KRT(dct):
    M,P_ = P(dct)
    print('original K')
    print(M)
    rot,k,trl = P2KRT(P_)
    print('new K')
    print(k)
    print('rot')
    print(rot)
    print('trl')
    print(trl)

In [21]:
dct = {'roll':0,'pitch':0,'yaw':60,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[ 0.5       -0.8660254  0.         2.       ]
 [ 0.8660254  0.5        0.         1.       ]
 [ 0.         0.         1.         0.       ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[ 0.5       -0.8660254  0.       ]
 [ 0.8660254  0.5        0.       ]
 [ 0.         0.         1.       ]]
trl
[[2.]
 [1.]
 [0.]]


In [22]:
dct = {'roll':0,'pitch':60,'yaw':60,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[ 0.25      -0.8660254  0.4330127  2.       ]
 [ 0.4330127  0.5        0.75       1.       ]
 [-0.8660254  0.         0.5        0.       ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[ 0.25      -0.8660254  0.4330127]
 [ 0.4330127  0.5        0.75     ]
 [-0.8660254  0.         0.5      ]]
trl
[[2.]
 [1.]
 [0.]]


In [23]:
dct = {'roll':60,'pitch':60,'yaw':60,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[ 0.25       -0.0580127   0.96650635  2.        ]
 [ 0.4330127   0.89951905 -0.0580127   1.        ]
 [-0.8660254   0.4330127   0.25        0.        ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[ 0.25       -0.0580127   0.96650635]
 [ 0.4330127   0.89951905 -0.0580127 ]
 [-0.8660254   0.4330127   0.25      ]]
trl
[[2.]
 [1.]
 [0.]]


In [24]:
dct = {'roll':0,'pitch':0,'yaw':120,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[-0.5       -0.8660254  0.         2.       ]
 [ 0.8660254 -0.5        0.         1.       ]
 [ 0.         0.         1.         0.       ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[-0.5       -0.8660254  0.       ]
 [ 0.8660254 -0.5        0.       ]
 [ 0.         0.         1.       ]]
trl
[[2.]
 [1.]
 [0.]]


In [25]:
dct = {'roll':180,'pitch':0,'yaw':120,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[-5.00000000e-01  8.66025404e-01  1.06057524e-16  2.00000000e+00]
 [ 8.66025404e-01  5.00000000e-01  6.12323400e-17  1.00000000e+00]
 [ 0.00000000e+00  1.22464680e-16 -1.00000000e+00  0.00000000e+00]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[-0.5        0.8660254  0.       ]
 [ 0.8660254  0.5        0.       ]
 [ 0.         0.        -1.       ]]
trl
[[2.]
 [1.]
 [0.]]


In [26]:
dct = {'roll':0,'pitch':0.5,'yaw':0,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[ 0.99996192  0.          0.00872654  2.        ]
 [ 0.          1.          0.          1.        ]
 [-0.00872654  0.          0.99996192  0.        ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[ 0.99996192  0.          0.00872654]
 [ 0.          1.          0.        ]
 [-0.00872654  0.          0.99996192]]
trl
[[2.]
 [1.]
 [0.]]


In [27]:
dct = {'roll':0,'pitch':60,'yaw':0,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[ 0.5        0.         0.8660254  2.       ]
 [ 0.         1.         0.         1.       ]
 [-0.8660254  0.         0.5        0.       ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]
trl
[[2.]
 [1.]
 [0.]]


In [28]:
dct = {'roll':60,'pitch':0,'yaw':0,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[ 1.         0.         0.         2.       ]
 [ 0.         0.5       -0.8660254  1.       ]
 [ 0.         0.8660254  0.5        0.       ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[ 1.         0.         0.       ]
 [ 0.         0.5       -0.8660254]
 [ 0.         0.8660254  0.5      ]]
trl
[[2.]
 [1.]
 [0.]]


In [29]:
dct = {'roll':0.5,'pitch':0.5,'yaw':0.5,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[ 0.99992385 -0.00865005  0.00880202  2.        ]
 [ 0.0087262   0.99992451 -0.00865005  1.        ]
 [-0.00872654  0.0087262   0.99992385  0.        ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[ 0.99992385 -0.00865005  0.00880202]
 [ 0.0087262   0.99992451 -0.00865005]
 [-0.00872654  0.0087262   0.99992385]]
trl
[[2.]
 [1.]
 [0.]]


In [30]:
dct = {'roll':60,'pitch':45,'yaw':30,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[ 0.61237244  0.28033009  0.73919892  2.        ]
 [ 0.35355339  0.73919892 -0.5732233   1.        ]
 [-0.70710678  0.61237244  0.35355339  0.        ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[ 0.61237244  0.28033009  0.73919892]
 [ 0.35355339  0.73919892 -0.5732233 ]
 [-0.70710678  0.61237244  0.35355339]]
trl
[[2.]
 [1.]
 [0.]]


In [31]:
dct = {'roll':60,'pitch':45,'yaw':290,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[ 0.24184476  0.67929002 -0.6928753   2.        ]
 [-0.66446302 -0.40443179 -0.62842964  1.        ]
 [-0.70710678  0.61237244  0.35355339  0.        ]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[ 0.24184476  0.67929002 -0.6928753 ]
 [-0.66446302 -0.40443179 -0.62842964]
 [-0.70710678  0.61237244  0.35355339]]
trl
[[2.]
 [1.]
 [0.]]


In [32]:
dct = {'roll':360,'pitch':245,'yaw':290,
       'tx':2,'ty':1,'tz':0,
       'f':0.1,'ku':5,'kv':2,'u0':0.1,'v0':0.2}
checkProcessP2KRT(dct)

[[-1.44543958e-01  9.39692621e-01 -3.09975519e-01  2.00000000e+00]
 [ 3.97131262e-01  3.42020143e-01  8.51650740e-01  1.00000000e+00]
 [ 9.06307787e-01  1.03511620e-16 -4.22618262e-01  0.00000000e+00]]
original K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
new K
[[0.5 0.  0.1]
 [0.  0.2 0.2]
 [0.  0.  1. ]]
rot
[[-0.14454396  0.93969262 -0.30997552]
 [ 0.39713126  0.34202014  0.85165074]
 [ 0.90630779  0.         -0.42261826]]
trl
[[2.]
 [1.]
 [0.]]
